In [83]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
import category_encoders as ce #pip install category_encoders

In [84]:
train = pd.read_csv('train.csv')
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,1
1,4,NaN,NaN,forest fire near la rong sask canada,1
2,5,NaN,NaN,al resid ask to shelter in place are be notifi...,1
3,6,NaN,NaN,number peopl receiv wildfir evacu order in cal...,1
4,7,NaN,NaN,just got se not thi photo from rubi alaska as ...,1
5,8,NaN,NaN,rockyfir updat california hwi number close in ...,1
6,10,NaN,NaN,flood disast heavi rain caus flash flood of st...,1
7,13,NaN,NaN,i am on top of the hil and i can see a fire in...,1
8,14,NaN,NaN,there is an emerg evacu happen now in the buil...,1
9,15,NaN,NaN,i am afraid that the tornado is come to our area,1


In [85]:
pos_col_text = train.columns.get_loc('text')+1
train.insert(loc=pos_col_text, column='len_text', value=train['text'].iloc[:].str.len())
train.head(10)

,id,keyword,location,text,len_text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,63,1
1,4,NaN,NaN,forest fire near la rong sask canada,37,1
2,5,NaN,NaN,al resid ask to shelter in place are be notifi...,108,1
3,6,NaN,NaN,number peopl receiv wildfir evacu order in cal...,54,1
4,7,NaN,NaN,just got se not thi photo from rubi alaska as ...,84,1
5,8,NaN,NaN,rockyfir updat california hwi number close in ...,96,1
6,10,NaN,NaN,flood disast heavi rain caus flash flood of st...,81,1
7,13,NaN,NaN,i am on top of the hil and i can see a fire in...,56,1
8,14,NaN,NaN,there is an emerg evacu happen now in the buil...,66,1
9,15,NaN,NaN,i am afraid that the tornado is come to our area,49,1


In [86]:
ce_bin_keyword = ce.BinaryEncoder(cols = ['keyword'], drop_invariant=True)
keyword_encoded = ce_bin_keyword.fit_transform(train['keyword'])

In [87]:
keyword_encoded

,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8
0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,1
7609,0,0,0,0,0,0,0,1
7610,0,0,0,0,0,0,0,1
7611,0,0,0,0,0,0,0,1


In [88]:
ce_bin_location = ce.BinaryEncoder(cols = ['location'], drop_invariant=True)
location_encoded = ce_bin_location.fit_transform(train['location'])

In [90]:
location_encoded = location_encoded.drop('location_12', axis=1)

In [92]:
text_encoded = pd.read_csv('train_text.csv')

In [93]:
aux = keyword_encoded.join(other=location_encoded, how='inner')
aux = text_encoded.join(other=aux, how='inner')
train_set = aux.join(other=train[['id', 'len_text', 'target']], how='inner', lsuffix='_')

In [94]:
train_set.head(10)

,sum,max,min,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,...,location_5,location_6,location_7,location_8,location_9,location_10,location_11,id,len_text,target
0,2.539967,0.504012,0.206936,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,63,1
1,2.387767,0.520764,0.302012,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,37,1
2,3.925010,0.585124,0.153758,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,108,1
3,2.515355,0.574226,0.150085,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6,54,1
4,3.069700,0.436824,0.179350,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7,84,1
5,3.196590,0.415266,0.100400,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,96,1
6,3.652190,0.421539,0.211922,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10,81,1
7,1.412882,0.737116,0.675766,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13,56,1
8,2.220835,0.528860,0.395912,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,14,66,1
9,1.966376,0.639305,0.391599,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,15,49,1


### XGBoost

In [95]:
from sklearn.model_selection import train_test_split

X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Parámetros que mejor funcionaron con pruebas a mano

In [96]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.800066


Best params de random forest

In [97]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 500)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.822729


### Test

In [98]:
test_set = pd.read_csv('test.csv')

In [99]:
pos_col_text = test_set.columns.get_loc('text')+1
test_set.insert(loc=pos_col_text, column='len_text', value=test_set['text'].iloc[:].str.len())
test_set.head(10)

,id,keyword,location,text,len_text
0,0,NaN,NaN,just happen a teribl car crash,31
1,2,NaN,NaN,heard about earthquak is differ not citi stay ...,59
2,3,NaN,NaN,there is a forest fire at spot pond gees are f...,89
3,9,NaN,NaN,apocalyps light spokan wildfir,31
4,11,NaN,NaN,typhoon soudelor kil number in china and taiwan,48
5,12,NaN,NaN,we are shake it is an earthquak,32
6,21,NaN,NaN,they would probabl stil show more life than ar...,70
7,22,NaN,NaN,hey how are you,16
8,27,NaN,NaN,what a nice hat,16
9,29,NaN,NaN,fuck off,9


In [100]:
ce_bin_keyword = ce.BinaryEncoder(cols = ['keyword'], drop_invariant=True)
keyword_encoded = ce_bin_keyword.fit_transform(test_set['keyword'])

In [101]:
keyword_encoded

,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8
0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
3258,0,0,0,0,0,0,0,1
3259,0,0,0,0,0,0,0,1
3260,0,0,0,0,0,0,0,1
3261,0,0,0,0,0,0,0,1


In [102]:
ce_bin_location = ce.BinaryEncoder(cols = ['location'], drop_invariant=True)
location_encoded = ce_bin_location.fit_transform(test_set['location'])

In [103]:
text_encoded = pd.read_csv('test_text.csv')

In [104]:
aux = keyword_encoded.join(other=location_encoded, how='inner')
aux = text_encoded.join(other=aux, how='inner')
test_set_encoded = aux.join(other=test_set[['id', 'len_text']], how='inner', lsuffix='_')

In [105]:
test_set_encoded

,sum,max,min,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,...,location_4,location_5,location_6,location_7,location_8,location_9,location_10,location_11,id,len_text
0,1.979137,0.615103,0.424578,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,31
1,2.638767,0.426320,0.342961,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,59
2,3.217465,0.410331,0.181811,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,3,89
3,1.958828,0.662541,0.407703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,9,31
4,2.346990,0.524219,0.151011,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,11,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,2.968900,0.595414,0.271113,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,10861,45
3259,4.407768,0.321931,0.140259,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,10865,135
3260,2.101836,0.523738,0.119779,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,10868,39
3261,2.501670,0.479922,0.093384,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,10874,41


In [106]:
preds = xg_reg.predict(test_set_encoded)
preds

array([1, 1, 1, ..., 1, 1, 1])

In [107]:
test_set['target'] = preds
test_set[['id', 'target']]

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


### Resultado

In [108]:
test_set[['id', 'target']].to_csv('result_xgb.csv', header=True, index=False)